# Welcome to Sales Report
author: Jack Lau <br>
last edit: 05/11/19 <br>
purpose: this report is use to show the sales info for the intopia

In [1]:
import pandas as pd
from intopia_analysis import *

In [93]:
df_contact = pd.read_csv('Intopia - Contact List - Sheet1.csv')
#df_sales = get_sales_data('game/period3/')
df_sales = get_sales_data('game/period6/')
df_production = get_production_data('game/period3/')
df_inventory_begin = get_inventory_data('game/period4/')
df_inventory_end = get_inventory_data('game/period6/')
df_ad = get_advertising_data('game/period3/')

In [43]:
df_inventory_end[df_inventory_end['Company'] == 9]

,Company,Units,Grade,type,region
5,9,911.616,0,x,ec
0,9,630.875,2,x,ec
0,9,886.988,1,x,wc
4,9,1085.086,0,x,wc
0,9,833.784,2,x,cc
5,9,911.616,0,x,cc
1,9,18.126,1,y,wc


In [44]:
pd.DataFrame(df_inventory.groupby(['Company','type','Grade'])['Units'].sum()).sort_values(by='Units', ascending=False).shape

NameError: name 'df_inventory' is not defined

In [61]:
df_inventory_end.groupby(['type', 'region','Grade'])['Units'].sum()

type  region  Grade
x     cc      0          360.500
              1         2760.369
              2         1043.069
              3         2996.967
              4         6572.000
      ec      0          902.564
              1         1192.810
              2          235.070
              3         4636.696
              4        11507.000
      wc      0         2491.741
              1          763.741
              2          360.298
              3         7186.042
              4        11065.000
              5          310.000
y     cc      0          520.000
              1          128.856
              2          260.000
              3         1416.894
              4         2468.000
      ec      0          346.483
              1            7.847
              3         2486.296
              4         2694.000
      wc      1          144.823
              2          585.030
              3         3719.165
              4         3120.849
              5        

In [94]:
df_ad_fixed = df_ad[['Company', 'type', 'region', 'Total']]
price = (df_sales['Price'].replace( '[\$,)]','', regex=True )
                   .replace( '[(]','-',   regex=True ).astype(float))
df_sales['Price'] = price

In [ ]:
df_supply = pd.merge(df_inventory, df_production, on=['Company', 'Grade', 'region', 'type'], how='outer')
df_supply = df_supply[['Company', 'type', 'Grade', 'region', 'Units', 'Unit Production']]
df_supply = df_supply.rename(columns={'Units':"inventory", 'Unit Production':"production"})
df_total = pd.merge(df_supply, df_sales, on=['Company', 'Grade', 'region', 'type'], how='outer')
df_total = df_total.rename(columns={'Unit Sales':"sales"})
df_total = pd.merge(df_total, df_ad_fixed, on=['Company', 'region', 'type'], how='outer')
df_total = df_total.rename(columns={'Total':"marketing"})

In [ ]:
df_total.groupby(['type', 'region','Grade'])['production', 'inventory'].sum().to_csv('total_supply.csv')

## Total Sales per area

In [95]:
df_sales['cost'] = df_sales['Price'] * df_sales['Unit Sales']

In [97]:
df_sales[(df_sales['Company'] == 17)]

,Company,Unit Sales,Grade,Price,type,region,cost
4,17,216.713,4,98.0,x,cc,21237.874
5,17,277.787,6,139.0,x,cc,38612.393
3,17,150.000,4,98.0,x,ec,14700.000
4,17,300.000,6,129.0,x,ec,38700.000
1,17,150.000,4,98.0,x,wc,14700.000
2,17,300.000,6,129.0,x,wc,38700.000
4,17,240.000,5,419.0,y,cc,100560.000
5,17,470.000,6,529.0,y,cc,248630.000
5,17,150.000,5,399.0,y,ec,59850.000
6,17,240.000,6,419.0,y,ec,100560.000


In [89]:
df_sales

,Company,Unit Sales,Grade,Price,type,region,cost
0,3,200.000,5,102.0,x,cc,20400.000
1,9,468.424,2,82.0,x,cc,38410.768
2,13,247.208,3,91.0,x,cc,22495.928
3,17,283.591,4,99.0,x,cc,28075.509
4,17,150.000,5,75.0,x,cc,11250.000
5,19,59.042,3,100.0,x,cc,5904.200
6,19,159.463,4,110.0,x,cc,17540.930
7,22,504.104,4,90.0,x,cc,45369.360
8,26,267.092,4,79.0,x,cc,21100.268
9,27,116.001,2,75.0,x,cc,8700.075


In [96]:
df_sales_sum = pd.DataFrame(df_sales.groupby(['type', 'region','Grade'])['Unit Sales'].sum())
df_sales_cost = pd.DataFrame(df_sales.groupby(['type', 'region','Grade'])['cost'].sum())
df_sales_insight = pd.merge(df_sales_sum, df_sales_cost, on=['type', 'region', 'Grade'], how='outer')
df_sales_insight['average price'] = df_sales_insight['cost'] / df_sales_insight['Unit Sales']
df_sales_insight

Unit Sales         cost  average price
type region Grade                                        
x    cc     2         416.527    31239.525      75.000000
            3         122.515    11026.350      90.000000
            4        1740.921   165062.999      94.813607
            5         920.916    89501.785      97.187784
            6        4870.834   594610.682     122.075743
     ec     3          37.328     2612.960      70.000000
            4         967.907    78828.700      81.442432
            5         180.985    19773.498     109.254900
            6        3291.889   392847.801     119.338107
     wc     3         625.081    43238.925      69.173315
            4        2018.068   172890.198      85.671146
            5         287.756    28878.446     100.357407
            6        5233.665   650452.533     124.282416
            7          55.000    11000.000     200.000000
y    cc     3         245.000    66150.000     270.000000
            4         479.328   150730.740     314.462623
            5        2664.884  1028318.975     385.877575
            6        5225.423  2444057.120     467.724263
     ec     3           0.096       21.120     220.000000
            4         476.783   144784.665     303.669940
            5         704.514   264165.600     374.961463
            6        2472.416  1019559.492     412.373764
     wc     3         168.000    47040.000     280.000000
            4         489.589   157413.415     321.521552
            5        1767.681   668021.285     377.908279
            6        3974.224  1751290.063     440.662143
            7          87.107    60974.900     700.000000

## Total marketing as percentage of sales

In [12]:
df_sales_per_team = pd.DataFrame(df_sales.groupby(['Company', 'region','type'])['cost'].sum())
df_sales_per_team = pd.merge(df_sales_per_team, df_ad_fixed, on=['type', 'region', 'Company'], how='outer')
df_sales_per_team['percentage_sales'] = df_sales_per_team['Total'] / df_sales_per_team['cost']
df_sales_per_team = df_sales_per_team[['Company', 'region', 'type', 'percentage_sales']]

In [14]:
df_sales_per_team

,Company,region,type,percentage_sales
0,9,cc,x,0.030080
1,9,ec,x,0.196323
2,9,wc,x,0.138155
3,14,ec,x,0.166667
4,14,wc,x,0.131752
5,17,cc,x,0.039683
6,17,ec,x,0.126251
7,17,wc,x,0.113434
8,19,cc,x,0.009926
9,19,ec,x,0.010659


In [13]:
df_total = pd.merge(df_total, df_sales_per_team, on=['Company', 'region', 'type'], how='outer')

NameError: name 'df_total' is not defined

## Top sales in CC

In [ ]:
df_total[(df_total['region'] == 'cc') & (df_total['type'] == 'x')].sort_values(by='sales', ascending=False)

In [ ]:
df_total[(df_total['region'] == 'cc') & (df_total['type'] == 'y')].sort_values(by='sales', ascending=False)

## Total Sales in WC 

In [ ]:
df_total[(df_total['region'] == 'wc') & (df_total['type'] == 'x')].sort_values(by='sales', ascending=False)

In [ ]:
df_total[(df_total['region'] == 'wc') & (df_total['type'] == 'y')].sort_values(by='sales', ascending=False)

## Total Sales in EC 

In [ ]:
df_total[(df_total['region'] == 'ec') & (df_total['type'] == 'x')].sort_values(by='sales', ascending=False)

In [ ]:
df_total[(df_total['region'] == 'ec') & (df_total['type'] == 'y')].sort_values(by='sales', ascending=False)